In [34]:
import pandas as pd
import numpy as np
import pylab as plt
import math
from IPython.display import display
from sklearn import preprocessing
import scipy.sparse as sp
from sklearn.feature_extraction.text import *
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
%matplotlib inline

RAND = 228
MAX_N = 10**4
MAX_TEST = 8

In [35]:
test_cases = []
MEAN_SQRT = 0
for i in range(1, MAX_TEST + 1):
    line_to_floats = lambda x: \
        np.array(list(map(
            float, filter(
                lambda line: len(line) > 0, x.split(' ')
            )
        )))
    
    inp = 'input00' + str(i) + '.txt'
    out = 'output00' + str(i) + '.txt'
    X = []
    y = []
    with open(inp, 'r') as file:
        for line in file:
            X.append(line_to_floats(line))
            
    with open(out, 'r') as file:
        for line in file:
            y.append(line_to_floats(line))
    y = y[1:]
    test_cases.append((np.array(X), np.array(y)))
    for p in y:
            MEAN_SQRT += p[0] * p[0] + p[1] * p[1]
print(MEAN_SQRT / MAX_TEST / MAX_N)

0.368255420893


In [36]:
def task_score(true, predict):
    serr = 0
    s = 0
    for t, p in zip(true, predict):
        er = t - p
        serr += er.dot(er)
        s += t.dot(t)
    return -10 * np.log10(serr / s) / 80 * 100

In [37]:
import random

def generate_seq(k):
    X_test, y_test = test_cases[6] #test_cases[random.randint(0, MAX_TEST - 1)]
    start = random.randint(0, MAX_N - 100)
    return X_test[start:start + k], y_test[start:start + k]
generate_seq(3)

(array([[-0.71528376, -1.55259913,  0.31971571,  0.0327823 ,  1.        ],
        [-0.70676562, -1.75088511,  0.42206614, -0.01754525,  1.        ],
        [-1.05059423, -1.77695944,  0.38969106, -0.10004758,  1.        ]]),
 array([[ 0.32058784,  0.03392245],
        [ 0.42253777, -0.01780685],
        [ 0.39291539, -0.10053026]]))

In [38]:
def genearte_X_y(n, k):
    X, y = [], []
    for i in range(n):
        test, answer = generate_seq(k)
        X.append(test[:-1].ravel())
        y.append(answer[-1])
    return np.array(X), np.array(y)

In [39]:
X_data, y_data = genearte_X_y(10000, 20)
display(X_data.shape)

(10000, 95)

In [40]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit_transform(X_data) #X_data #PolynomialFeatures(degree=2).fit_transform(X_data)
y = y_data
display(X.shape)
display(X)

(10000, 95)

array([[-0.05221089, -0.37827851,  0.00327269, ...,  0.02423569,
         0.03722495, -1.2920968 ],
       [ 0.82119937, -0.31613295,  0.05753672, ...,  0.96907843,
        -0.12132654,  0.77393582],
       [-0.60989462,  2.34942947,  0.00327269, ...,  0.02423569,
         0.03722495, -1.2920968 ],
       ..., 
       [ 0.92755765, -0.91700983, -0.96820836, ...,  0.47249136,
        -0.04785397,  0.77393582],
       [-0.99554476, -0.59611391, -0.61628057, ...,  0.02423569,
         0.03722495, -1.2920968 ],
       [ 0.03606619,  0.75022583, -0.65789385, ...,  0.02423569,
         0.03722495, -1.2920968 ]])

In [43]:
from sklearn.ensemble import RandomForestRegressor

kfl = KFold(random_state=RAND, n_splits=3, shuffle=True)

regr = RandomForestRegressor(
    n_estimators=300, 
    max_depth=3, 
    random_state=RAND#, 
    #max_features='sqrt'
)

In [44]:
cvs_regr = cross_val_score(regr, X, y, cv=kfl, scoring='neg_mean_squared_error')
display(np.mean(cvs_regr))

-0.11475921106913441

In [45]:
regr.fit(X, y)
result = regr.predict(X)
display(task_score(y, result))
display(list(zip(y, result)))

3.3294553611283169

[(array([ 0.36978106,  0.54791485]), array([ 0.13986609,  0.09800576])),
 (array([ 0.00739161,  0.44958007]), array([ 0.24576407,  0.07929465])),
 (array([ 0.49447641, -0.11413741]), array([ 0.13986609,  0.09800576])),
 (array([ 0.07280132,  0.49536923]), array([ 0.1366401 ,  0.15343318])),
 (array([ 0.52315156,  0.35636277]), array([ 0.1241721 ,  0.08711983])),
 (array([ 0.00149731,  0.66721214]), array([ 0.06943484,  0.20140715])),
 (array([ 0.3595352 , -0.46602234]), array([ 0.37032221, -0.48181786])),
 (array([ 0.83620537, -0.2906344 ]), array([ 0.14017671,  0.04437249])),
 (array([-0.06402005,  0.09480453]), array([ 0.1241721 ,  0.08711983])),
 (array([ 0.82718254,  0.37135576]), array([ 0.25717544,  0.12096537])),
 (array([-0.15278742, -0.15100308]), array([-0.02144819, -0.07961148])),
 (array([-0.53193681,  1.60395255]), array([-0.39015659,  0.40527021])),
 (array([-0.14788947, -0.15344591]), array([-0.05398966, -0.01011942])),
 (array([ 0.10808628, -0.03851491]), array([ 0.1401

In [ ]:
from sklearn.svm import SVR

class multi_model:
    def __init__(self, model, *args, **param):
        self.X_m = model(*args, **param)
        self.Y_m = model(*args, **param)
    def fit(self, X, y):
        y_y, y_x = zip(*y)
        self.X_m.fit(X, y_x)
        self.Y_m.fit(X, y_y)
    def predict(self, X):
        return list(zip(self.X_m.predict(X), self.Y_m.predict(X)))
    
svr = multi_model(SVR, kernel='poly')

In [ ]:
svr.fit(X, y)
result = svr.predict(X)
display(task_score(y, result))
display(list(zip(y, result)))